In [ ]:
!pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip

def split_video_audio(video_path, audio_output_path, video_output_path):
    """
    Splits a video into audio and video components.

    Args:
        video_path: Path to the input video file.
        audio_output_path: Path to save the extracted audio.
        video_output_path: Path to save the video without audio.
    """
    try:
        video = VideoFileClip(video_path)

        # Extract audio
        video.audio.write_audiofile(audio_output_path)

        # Extract video without audio
        video.without_audio().write_videofile(video_output_path, codec="libx264")  # You may need to adjust the codec

        video.close()  #Important to properly release resources.
    except Exception as e:
        print(f"Error: {e}")

# Example usage:
video_file = "input.mp4"  # Replace with your video file
audio_file = "audio.mp3"
video_no_audio_file = "video_no_audio.mp4"

split_video_audio(video_file, audio_file, video_no_audio_file)
print("Splitting Completed")

In [ ]:
import skvideo.io
import scipy.io.wavfile as wav
import numpy as np

def video_to_audio_frames(video_path, audio_path):
    try:
        videodata = skvideo.io.vread(video_path)
        FPS = skvideo.io.vread(video_path, as_dict=True)["rate"]
        audiodata = skvideo.io.vread(video_path, as_dict=True)["audio"]
        SAMPLE_RATE = 44100
        wav.write(audio_path, SAMPLE_RATE, audiodata)
        return FPS
    except Exception as e:
        print(f"Error: {e}")

def video_to_frames(video_path, output_path="frame_%d.jpg"):
    try:
        videodata = skvideo.io.vread(video_path)
        skvideo.io.vwrite(output_path, videodata)
    except Exception as e:
        print(f"Error: {e}")
#Example Usage
video_to_audio_frames("input.mp4", "Audio.wav") # Returns the Frames per second as well
video_to_frames("input.mp4", "frames%d.jpg") #Extracts the frames into the name specified. 

In [ ]:
from moviepy.editor import VideoFileClip
from PIL import Image

def video_to_audio_frames(video_path, audio_path):
    try:
        video = VideoFileClip(video_path)
        video.audio.write_audiofile(audio_path)
        return video.fps
    except Exception as e:
        print(f"Error: {e}")

def video_to_frames(video_path, output_path="frame_%d.jpg"):
    try:
        video = VideoFileClip(video_path)
        for i, frame in enumerate(video.iter_frames()):
            frame_image = Image.fromarray(frame)
            frame_image.save(output_path % i)
    except Exception as e:
        print(f"Error: {e}")

# Example Usage
fps = video_to_audio_frames("input.mp4", "Audio.wav")  # Returns the Frames per second as well
video_to_frames("input.mp4", "frames%d.jpg")  # Extracts the frames into the name specified.

In [ ]:
import skvideo.io
import scipy.io.wavfile as wav
import numpy as np

# Monkey patch numpy to remove the deprecated alias:
if hasattr(np, 'float'):
    delattr(np, 'float')  

def video_to_audio_frames(video_path, audio_path):
    try:
        videodata = skvideo.io.vread(video_path)
        FPS = skvideo.io.vread(video_path, as_dict=True)["rate"]
        audiodata = skvideo.io.vread(video_path, as_dict=True)["audio"]
        SAMPLE_RATE = 44100
        wav.write(audio_path, SAMPLE_RATE, audiodata)
        return FPS
    except Exception as e:
        print(f"Error: {e}")

def video_to_frames(video_path, output_path="frame_%d.jpg"):
    try:
        videodata = skvideo.io.vread(video_path)
        skvideo.io.vwrite(output_path, videodata)
    except Exception as e:
        print(f"Error: {e}")

# Example usage
video_to_audio_frames("/content/drive/MyDrive/Datasets/dataset/1-1004/A.Beautiful.Mind.2001__#00-01-45_00-02-50_label_A.mp4", "Audio.wav") 
video_to_frames("/content/drive/MyDrive/Datasets/dataset/1-1004/A.Beautiful.Mind.2001__#00-01-45_00-02-50_label_A.mp4", "frames%d.jpg")

In [ ]:
from moviepy.editor import VideoFileClip
import skvideo.io
import os

def video_to_audio_frames(video_path, audio_path):
    try:
        # Extract audio using moviepy
        video = VideoFileClip(video_path)
        video.audio.write_audiofile(audio_path)
        FPS = video.fps
        return FPS
    except Exception as e:
        print(f"Error extracting audio: {e}")

def video_to_frames(video_path, output_path="frame_%d.jpg"):
    try:
        # Extract frames using skvideo
        videodata = skvideo.io.vread(video_path)
        if not os.path.exists(os.path.dirname(output_path)):
            os.makedirs(os.path.dirname(output_path))
        for i, frame in enumerate(videodata):
            frame_path = output_path % i
            skvideo.io.vwrite(frame_path, frame)
    except Exception as e:
        print(f"Error extracting frames: {e}")

# Example usage
video_to_audio_frames("/content/drive/MyDrive/Datasets/dataset/1-1004/A.Beautiful.Mind.2001__#00-01-45_00-02-50_label_A.mp4", "Audio.wav") 
video_to_frames("/content/drive/MyDrive/Datasets/dataset/1-1004/A.Beautiful.Mind.2001__#00-01-45_00-02-50_label_A.mp4", "frames/frame_%d.jpg")

In [2]:
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.1/22.1 MB 1.9 MB/s eta 0:00:0000:0100:01


In [3]:
import av

def split_video_audio_pyav(video_path, audio_output_path, video_output_path):
    try:
        container = av.open(video_path)

        video_stream = next((s for s in container.streams if s.type == 'video'), None)
        audio_stream = next((s for s in container.streams if s.type == 'audio'), None)

        if not video_stream or not audio_stream:
            print("Video or audio stream not found.")
            return

        # --- Configure video encoder for hardware acceleration (example) ---
        video_codec_name = 'h264_nvenc'  # Adjust for your GPU
        video_output = av.open(video_output_path, mode='w', format='mp4')
        new_video_stream = video_output.add_stream(video_codec_name, rate=video_stream.average_rate)
        new_video_stream.width = video_stream.width
        new_video_stream.height = video_stream.height
        new_video_stream.pix_fmt = 'yuv420p' # Or another supported pixel format
        # options = {'tune': 'hq'} # For nvidia encoder, tune it to high quality
        # new_video_stream.options = options
        new_video_stream.codec_context.time_base = video_stream.time_base
        # --- End video encoder configuration ---

        #Configure audio
        audio_output = av.open(audio_output_path, mode="w", format="wav")
        new_audio_stream = audio_output.add_stream('pcm_s16le', rate=audio_stream.rate)
        new_audio_stream.layout = audio_stream.layout
        new_audio_stream.rate = audio_stream.rate
        new_audio_stream.codec_context.time_base = audio_stream.time_base

        for frame in container.decode(video_stream):
            new_video_stream.encode(frame)

        for frame in container.decode(audio_stream):
            new_audio_stream.encode(frame)

        # Flush Stream
        for packet in new_video_stream.encode():
            video_output.mux(packet)
        for packet in new_audio_stream.encode():
            audio_output.mux(packet)

        # Close the files
        video_output.close()
        audio_output.close()
        container.close()


    except Exception as e:
        print(f"Error: {e}")

In [13]:
video_path="input.mp4"
audio_output_path="audio.wav"
video_output_path="video_no_audio.mp4"
split_video_audio_pyav(video_path, audio_output_path, video_output_path)

In [9]:
import av

def split_video_audio_pyav(video_path, audio_output_path, video_output_path):
    try:
        container = av.open(video_path)

        video_stream = next((s for s in container.streams if s.type == 'video'), None)
        audio_stream = next((s for s in container.streams if s.type == 'audio'), None)

        if not video_stream or not audio_stream:
            print("Video or audio stream not found.")
            return

        # --- Configure video encoder for CPU only ---
        video_codec_name = 'libx264'  # Use libx264 for CPU-based H.264 encoding
        video_output = av.open(video_output_path, mode='w', format='mp4')
        new_video_stream = video_output.add_stream(video_codec_name, rate=video_stream.average_rate)
        new_video_stream.width = video_stream.width
        new_video_stream.height = video_stream.height
        new_video_stream.pix_fmt = 'yuv420p'  # Or another supported pixel format
        new_video_stream.codec_context.time_base = video_stream.time_base

        # --- End video encoder configuration ---

        # Configure audio. Crucially, open the output file *before* creating the stream!
        audio_output = av.open(audio_output_path, mode="w", format="wav")
        new_audio_stream = audio_output.add_stream('pcm_s16le', rate=audio_stream.rate)  # PCM for WAV
        new_audio_stream.layout = audio_stream.layout #Layout may differ, depending on audio channels
        new_audio_stream.rate = audio_stream.rate
        new_audio_stream.codec_context.time_base = audio_stream.time_base

        for frame in container.decode(video_stream):
            for packet in new_video_stream.encode(frame):
                video_output.mux(packet)

        for frame in container.decode(audio_stream):
            for packet in new_audio_stream.encode(frame):
                audio_output.mux(packet)

        # Flush Stream (Important!)
        for packet in new_video_stream.encode():
            video_output.mux(packet)
        for packet in new_audio_stream.encode():
            audio_output.mux(packet)

        # Close the files (Important!)
        video_output.close()
        audio_output.close()
        container.close()

    except Exception as e:
        print(f"Error: {e}")

In [12]:
import cv2

def split_video_audio_opencv(video_path, audio_output_path, video_output_path):
    try:
        video = cv2.VideoCapture(video_path)
        fps = video.get(cv2.CAP_PROP_FPS)
        width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

        # --- Write Video (No Audio) ---
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec (adjust as needed)
        out = cv2.VideoWriter(video_output_path, fourcc, fps, (width, height))

        # --- Extract Frames and Write ---
        for i in range(frame_count):
            ret, frame = video.read()
            if not ret:
                break
            out.write(frame)
        # --- Release resources ---
        video.release()
        out.release()
        # Extract audio using moviepy, or another audio extraction method
        from moviepy.editor import VideoFileClip
        video = VideoFileClip(video_path)
        video.audio.write_audiofile(audio_output_path)
        video.close()
    except Exception as e:
        print(f"Error: {e}")